## Find neighbours of atoms in PDB structures using biopython

The PDB structure contains the (x, y, z) coordinates of every atom in the structure. biopython has an implementation of a fast algorithm, `NeighborSearch`, that can find neighbouring atoms that are within a certain distance of a given coordinate.

In [1]:
import os.path
from Bio.PDB import PDBParser, NeighborSearch

Define the directory where the Chothia renumbered PDB file are stored.

In [2]:
PDB_DIR = "../data/pdbs"

Given the `pdb_id`, find the path to the corresponding PDB file

In [3]:
pdb_id = "9ds1"
filename = os.path.join(PDB_DIR, f"{pdb_id}_chothia.pdb")

First, we need to parse the PDB file

In [4]:
parser = PDBParser(PERMISSIVE=1)
structure = parser.get_structure(pdb_id, filename)

Then we need to initialise the data structure that is used to perform the search of neighbours. Here we use all atoms of the structure for demonstration. This can be wasteful. Later, it can be advantageous to only use the atoms of the antigen, for example.

In [19]:
atoms = list(structure.get_atoms())
ns = NeighborSearch(atoms)

Now we can use the `ns.search(coords, distance)` method to find all atoms that are within `distance` angstroms from the position specified by the `[x, y, z]` array `coords`.

We can retrieve the position of an atom using its `coord` property:

In [23]:
atom = structure[0]['H'][52]['N']
atom.coord

array([ 14.805, -56.923,  25.988], dtype=float32)

To find all atoms in vicinity of an atom we simply pass the coordinates of the atom to the `ns.search` method.

In [24]:
close_atoms = ns.search(atom.coord, 4.0)

`close_atoms` now contains the atom entities of all atoms that are within 4.0 angstrom of the supplied coordinates. To find the residues and chains these atoms belong to, we can either use repeated application of the `get_parent()` method, or we can simply use the `get_full_id()` method.

As we want the output as a DataFrame, we first create a vector of dicts, and then create the DataFrame. 

In [ ]:
import pandas as pd

data = []

for atom in close_atoms:
    s, m, c, r, a = atom.full_id
    resnum = str(r[1]) + r[2]

    # to get the resname we need to access the residue the atom belongs to
    residue = atom.get_parent()
    resname = residue.get_resname()
    data.append(dict(chain = c, resnum = resnum, resname = resname, atom = a[0]))


df = pd.DataFrame(data)

df

Now we have learned how to find all atoms within the vicinity of a given atom. 

We can use this to find the atomic contact points between residues of an antibody and residues of its bound antigen.
We say that the residue of an antibody is in contact with the residue of an antigen if the distance 
of the corresponding atoms is less that a threshold distance. Often a threshold of 4 angstroms is used.

Write a function `atomic_contact_points(ab_chain, ag_chain, distance)` that loops over the atoms in ab_chain to get all atoms of ag_chain that are within distance, and reports a DataFrame with columns
- ab_resnum
- ab_icode
- ab_resname
- ab_atom
- ag_resnum
- ag_icode
- ag_resname
- ag_atom

But only report for residues that are amino acids, i.e. het_flag == ' '.

In [ ]:
def atomic_contact_points(ab_chain, ag_chain, distance):
    ...

Now let's test the function

In [ ]:
cp = atomic_contact_points(structure[0]['H'], structure[0]['A'], 4.0)
cp

Now we have a DataFrame with atomic contact points. Often we are interested in residue contact points. These can be obtained from the DataFrame using  (check the pandas grouping documentation).

For, example, to get the number of atom-atom and residue-residue contacts per antibody residue, we can use

In [ ]:
cp.groupby(['ab_resnum', 'ab_resname']).agg(natomcontacts = ('ag_resnum', 'size'),
                                            nrescontacts = ('ag_resnum', 'nunique')).reset_index()

We are also interested in the occurrence of residue numbers in the heavy and light chains.
Write a function `residue_occurrence(chain)` that creates as output a DataFrame with columns 
- ab_resnum
- ab_icode
- ab_resname

Restrict to het_name == ' ' and resnum <= 128

In [ ]:
def residue_occurrence(chain):

    ...

Let's test the function

In [ ]:
ro = residue_occurrence(structure[0]['H'])
ro

In the next notebook we are going to loop over all PDB files. It will, therefore, be necessary to add some more columns to the DataFrames. We can add a column at a given position using the `insert()` method. For example,

In [ ]:
ro.insert(loc = 0, column = 'chain_type', value = 'heavy')
ro.insert(loc = 0, column = 'pdb_id', value = pdb_id)
ro